# mdmanager

> Obsidian App Compatible Markdown reader

In [ ]:
#| default_exp mdmanager

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from memexplatform_obsidian.settings import ObsidianConfig
from memexplatform_obsidian.commons import ExtensionTypes, Page
from enum import Enum
import urllib
import mistletoe
from pathlib import Path
from mistletoe.ast_renderer import ASTRenderer
from mistletoe import Document
import yaml
import re
from typing import List
from mistletoe.block_token import BlockToken, Quote
from mistletoe import Document, HTMLRenderer
from mistletoe.ast_renderer import AstRenderer
from mistletoe import span_token, block_token, token
from mistletoe.span_token import Link, RawText, EscapeSequence, AutoLink
from mistletoe.span_tokenizer import tokenize
from mistletoe.token import Token
from memexplatform_obsidian.commons import MountPaths
from mistletoe.block_tokenizer import tokenize_block
import pathlib
import mimetypes
from typing import Optional
from datetime import datetime, timezone
import hashlib
from urllib.parse import urlparse
# from fasthtml.common import *
# from monsterui.all import *
import uuid
import json
from memexplatform_obsidian.engine.apexchart import ApexChartEngine


In [ ]:
#| export
EXT_MIME_OVERRIDES = {
    # --- Audio ---
    ".mp3": "audio/mpeg",
    ".m4a": "audio/mp4",
    ".aac": "audio/mp4",
    ".wav": "audio/wav",
    ".oga": "audio/ogg",
    ".ogg": "audio/ogg",   # audio context
    ".flac": "audio/flac",

    # --- Video ---
    ".mp4": "video/mp4",
    ".m4v": "video/mp4",
    ".webm": "video/webm",
    ".ogv": "video/ogg",
    ".mov": "video/quicktime",
    ".mkv": "video/webm",   # force fallback for browsers
}

def guess_mime(path: str) -> str:
    suffix = pathlib.Path(path).suffix.lower()
    if suffix in EXT_MIME_OVERRIDES:
        return EXT_MIME_OVERRIDES[suffix]
    mime = mimetypes.guess_type(path, strict=False)[0]
    return mime or "application/octet-stream"

In [ ]:
#| export
config = ObsidianConfig(); config
# (config.PORTAL/'index.qmd').exists()

ObsidianConfig(PREFIX='/obsidian', OBSIDIAN_VAULT=Path('/Users/rahul1.saraf/rahuketu/programming/notesobs'), OBSIDIAN_DB=Path('/Users/rahul1.saraf/rahuketu/programming/gitea/memexplatform_obsidian/data/memexplatform_obsidian.db'), ENABLE_DB_MODE=True)

In [ ]:
#| export
def get_subdirs(vault: Path):
    subdirs = []
    for p in vault.rglob("*"):
        if p.is_dir():
            # Check if any parent directory (including self) should be skipped
            if any(part.startswith((".", "_")) or part.startswith("logseq") for part in p.parts): continue
            subdirs.append(p)
    return subdirs

In [ ]:
#| export
def resolve_note_path(vault:Path, file:str) -> Path | None:
    search_term = urllib.parse.unquote(file); search_term
    rel_path = Path(search_term); rel_path
    fname = rel_path.name
    fldr = rel_path.parent; fldr
    pfname = None
    extensions = ('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.svg', '.webp', '.tiff', '.ico',
                    '.base', '.canvas', '.md', '.qmd', '.ipynb'
                )
    if fldr.name == "":
        if fname.lower().endswith(extensions):
            for sub in get_subdirs(vault):
                pfname = next(sub.rglob(fname), None)
                if pfname: break
        else:
            mdfname = fname +".md"
            for sub in get_subdirs(vault):
                pfname = next(sub.rglob(mdfname), None)
                if pfname: break
    else:
        if fname.lower().endswith(extensions):
            pfname = next((vault/fldr).rglob(fname), None)
        else:
            mdfname = fname +".md"
            pfname = next((vault/fldr).rglob(mdfname), None)
    if pfname: return pfname
    else: 
        lsfldr = vault/search_term
        if lsfldr.is_dir(): return lsfldr
        else: return None

In [ ]:
# config.OBSIDIAN_VAULT.ls()
vault_path = Path(config.OBSIDIAN_VAULT)
subdirs = get_subdirs(vault_path); subdirs
pages_dir = config.OBSIDIAN_VAULT/"pages"; pages_dir.ls()[-1]

doc_real = pages_dir/"Product Mindset for RAG.md"; doc_real.read_text()

'---\ntitle: Product Mindset for RAG\ntags:\ncollection:\n  - "[[pages/RAG|RAG]]"\n---\n## Introduction\n\nDoing correct things involves approaching the problem with the right mindset. Approaching RAG as a project with technical metrics like choosing vector embeddings, llm , chunk size often misses the point of building a RAG product in the first place, providing correct information to the user in shortest possible time. \n\nTo get things right we need to think of RAG as recommendation system and identify the ways to continuously and iteratively improve our system as we systematically collect and receive user feedback. Our main objective should be customer delight, retention and increased engagement. \n\nSo how do we get started. In a project mindset, most of the time people face issues with chicken and egg problem.  Without domain knowledge, without knowing anything about the users and their experience with your product; How do you get started? Should we spend huge amount of money in 

In [ ]:
doc = Document(["Hello *world*!"])
print(doc)  # token tree

<mistletoe.block_token.Document with 1 child line_number=1>


In [ ]:
m = "https://https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.vecteezy.com%2Ffree-png%2Fimage-icon&psig=AOvVaw1kpNQHn1o3--A8ed2ZVFST&ust=1757915995941000&source=images&cd=vfe&opi=89978449&ved=0CBUQjRxqFwoTCPDzxsXJ148DFQAAAAAdAAAAABAE"
pathlib.Path(m).suffix

'.com%2Ffree-png%2Fimage-icon&psig=AOvVaw1kpNQHn1o3--A8ed2ZVFST&ust=1757915995941000&source=images&cd=vfe&opi=89978449&ved=0CBUQjRxqFwoTCPDzxsXJ148DFQAAAAAdAAAAABAE'

In [ ]:
#| export

class ObsidianEmbed(Link):
    """
    Obsidian-style embeds:
      - ![[Note]]        → embed note or image
      - ![alt](target)   → embed note or image
      - ![alt](target "title") → embed with title

    Behavior:
      - If target ends with .md / .qmd / .canvas / .base → inline blockquote of rendered content
      - If target ends with image extension → inline <img>
      - Otherwise → fallback to simple link
    """

    # ![[Note|Alias]] OR ![alt](target "title")
    pattern = re.compile(
        r'!\[\[([^\]|]+)(?:\|([^\]]+))?\]\]'         # ![[Note|Alias]]
        r'|!\[([^\]]*)\]\(([^)\s]+)(?:\s+"([^"]+)")?\)'  # ![alt](target "title")
    )
    parse_inner = False
    parse_group = 0
    repr_attributes = ("ext", "fname", "children", "title", "label", "dest_type", "title_delimiter", "label", "src")

    def __init__(self, match):
        if match.group(1):  # ![[Note|Alias]]
            raw_target = EscapeSequence.strip(match.group(1).strip())
            alias = match.group(2) or raw_target
            title = alias
        else:  # ![alt](target "title")
            alias = match.group(3) or ""
            raw_target = EscapeSequence.strip(urllib.parse.unquote(match.group(4).strip()))
            title = match.group(5) or alias

        self.title = title
        if raw_target.startswith(("http://", "https://")):
            self.target = raw_target
            self.fname = None
            parsed_url = urlparse(raw_target)
            self.ext = pathlib.Path(parsed_url.path).suffix
            self.src = raw_target
        else:
            self.fname = raw_target
            self.ext = pathlib.Path(raw_target).suffix or ".md"
            self.target = MountPaths.embed.to(file=raw_target, ext=self.ext, title=self.title)
            self.src = self.target

        self.label = None
        self.dest_type = "obsidianembed"
        self.title_delimiter = None

        # Render children as the alias text
        self.children = list(span_token.tokenize_inner(alias))

In [ ]:
#| export

class ObsidianLink(Link):
    """
    Obsidian-style markdown link [Alias](target "title").
    - If target is an http(s) URL, render as normal.
    - Otherwise, rewrite to open?file=<escaped target>.
    """

    # Matches [Alias](target "title")
    pattern = re.compile(r'\[([^\]]+)\]\(([^)\s]+)(?:\s+"([^"]+)")?\)')
    parse_inner = False
    parse_group = 0

    repr_attributes = ("ext", "fname", "children", "title", "label", "dest_type", "title_delimiter", "label", "src")

    def __init__(self, match):
        alias = match.group(1).strip()
        raw_target = EscapeSequence.strip(urllib.parse.unquote(match.group(2).strip()))
        title = match.group(3) or alias

        self.title = title
        if raw_target.startswith(("http://", "https://")):
            self.target = raw_target
            self.fname = None
            self.ext = None
            self.src = raw_target
        else:
            self.fname = raw_target
            self.ext = pathlib.Path(raw_target).suffix or ".md"
            self.target = MountPaths.open.to(file=raw_target, title=self.title)
            self.src = self.target

        
        self.label = None
        self.dest_type = "obsidianlink"
        self.title_delimiter = None
        # self.children = [RawText(alias)]
        self.children = list(span_token.tokenize_inner(alias))


In [ ]:
#| export
class WikiLink(Link):
    """
    Obsidian-style wikilink [[Note|Alias]] that behaves like mistletoe.Link.
    """
    pattern = re.compile(r"\[\[([^\]|]+)(?:\|([^\]]+))?\]\]")
    parse_inner = False
    parse_group = 0
    repr_attributes = ("ext", "fname", "children", "title", "label", "dest_type", "title_delimiter", "label", "src")

    def __init__(self, match):
        # The "target" (like a URL)
        raw_target = EscapeSequence.strip(match.group(1).strip())
        self.target = MountPaths.open.to(file=raw_target)
        # Use alias if present, otherwise same as target
        self.alias = match.group(2) or raw_target
        self.fname = raw_target
        self.ext = pathlib.Path(raw_target).suffix or ".md"
        self.src = self.target

        # Fill in Link attributes
        self.title = self.alias    # Obsidian links don’t have a title
        self.label = None   # No label
        self.dest_type = "wikilink"
        self.title_delimiter = None

        # children: inline text tokens for the alias (like how Link holds link text)
        # self.children = [RawText(self.alias)]
        self.children = list(span_token.tokenize_inner(self.alias))

In [ ]:
#| export
class AnyLink(span_token.SpanToken):
    """
    Match any URI scheme like obsidian://, logseq://, code://, http://, https:// etc.
    Does NOT match markdown links [text](...) or wikilinks [[Note]].
    """
    # Must capture the whole URL in group 1
    pattern = re.compile(r'(?<![\]\)"])(([a-zA-Z][a-zA-Z0-9+.-]*://[^\s]+))')
    parse_group = 1
    repr_attributes = ("title", "target", "children")

    def __init__(self, match):
        url = match.group(1)
        self.children = (RawText(url),)
        self.target = url
        self.title = url

In [ ]:
#| export

class Properties(Token):
    repr_attributes = ("key", "children")

    def __init__(self, key, value):
        self.key = key
        # Parse value recursively into proper span tokens
        self.children = self._parse_value(value)

    def _parse_value(self, value):
        """Recursively parse value into a flat list of span tokens."""
        if isinstance(value, str):
            stripped = value.strip()
            # Detect plain URLs and convert to AutoLink
            m = AnyLink.pattern.match(stripped)
            if m: return [AnyLink(m)]

            # Otherwise parse as regular inline markdown
            return span_token.tokenize_inner(stripped)
            # return span_token.tokenize_inner(value)

        elif isinstance(value, list):
            tokens = []
            for v in value:
                tokens.extend(self._parse_value(v))
            return tokens

        elif isinstance(value, dict):
            tokens = []
            for k, v in value.items():
                tokens.extend(self._parse_value(v))
            return tokens

        else:
            # Fallback for non-string values
            return [RawText(str(value))]

In [ ]:
#| export
class TagLink(Link):
    """
    Obsidian-style tag link #tag that behaves like a mistletoe.Link.
    Renders to <a href="/tags/tag">#tag</a> or similar.
    """
    # Matches hashtags at word boundaries: # followed by word characters/dashes
    pattern = re.compile(r'(?<!\w)#([\w\-]+)')
    parse_group = 0
    parse_inner = False

    repr_attributes = ("fname", "children", "title", "label", "dest_type", "title_delimiter", "label")

    def __init__(self, match):
        tag_name = match.group(1)

        # URL target (you can change the prefix to match your app)
        self.target = MountPaths.tags.to(name=tag_name)
        self.title = f"#{tag_name}"
        self.label = None
        self.dest_type = "taglink"
        self.title_delimiter = None

        # Displayed text is "#tag"
        self.children = [RawText(f"#{tag_name}")]
        self.fname = f"#{tag_name}"

In [ ]:
#| export
class Frontmatter(BlockToken):
    """
    YAML Frontmatter token.
    This is a leaf block token that captures metadata enclosed by --- ... ---.
    """

    repr_attributes = BlockToken.repr_attributes + ("data",)

    # Match `---` followed by anything until the next `---` on its own line
    # pattern = re.compile(
    #     r'(?ms)^(?:---)\s*\n(.*?)(?:\n---\s*$)'
    # )

    pattern = re.compile(
        r'(?ms)\A---\s*\n(.*?)(?:\n---\s*\n)'
    )

    def __init__(self, content: str):
        self.content = content.strip()
        try:
            parsed = yaml.safe_load(self.content) or {}
        except yaml.YAMLError as e:
            parsed = {"error": str(e)}


        if isinstance(parsed, dict):
            self.data = parsed
        else:
            # wrap scalars/lists into a dict
            self.data = {"value": parsed}
        
        # Process the YAML data into children, parsing each value as markdown
        self.children = []
        
        for k, v in self.data.items():
            if k.lower() == "tags":
                tags = v if isinstance(v, list) else [v]
                            # Prepend "#" so later inline parsing makes TagLinks
                processed = [f"#{t}" for t in tags if isinstance(t, str)]
                self.children.append(Properties(k, processed))
            else: self.children.append(Properties(k, v))

    @classmethod
    def start(cls, line: str) -> bool:
        # Only starts if line is `---`
        return line.strip() == "---"

    @classmethod
    def check_interrupts_paragraph(cls, lines):
        return cls.start(lines.peek())

    @classmethod
    def read(cls, lines):
        if lines.line_number() > 1:
            return None 
        # Consume first line (`---`)
        next(lines)
        buffer = []
        for line in lines:
            if line.strip() == "---":
                break
            buffer.append(line)
        return "".join(buffer)

In [ ]:
#| export
CALLOUT_TYPE_MAP = {
    'info': 'info', 'tip': 'tip', 'warning': 'warning',
    'caution': 'warning', 'danger': 'danger', 'error': 'danger',
    'success': 'success', 'note': 'note', 'abstract': 'abstract',
    'todo': 'todo', 'question': 'question', 'help': 'question',
    'example': 'example', 'quote': 'quote', 'bug': 'bug', 'failure': 'failure',
}

# Updated regexes

# Full block: marker line + at least one more >-line:
CALLOUT_BLOCK_REGEX = re.compile(
    r'(?ms)\A'                                         # start
    r'>\s*\[!([A-Za-z0-9_-]+)\]([+-]?)(?:\s*(.*?))?'    # capture type, optional fold-flag, optional title (non-greedy)
    r'(?:\r?\n>.*)+'                                   # body: one or more quoted lines
)

# First line only, for start detection
CALLOUT_FIRST_LINE_REGEX = re.compile(
    r'^\s*>\s*\[!([A-Za-z0-9_-]+)\]([+-]?)(?:\s*(.*?))?$'
)

class ObsidianCallout(Quote):

    @staticmethod
    def start(line: str) -> bool:
        return bool(CALLOUT_FIRST_LINE_REGEX.match(line))

    @classmethod
    def read(cls, lines):
        raw_buffer = []
        start_line_number = lines.line_number()
        try:
            first_line = next(lines)
        except StopIteration:
            return None
        raw_buffer.append(first_line)

        for nl in lines:
            if nl.lstrip().startswith('>'):
                raw_buffer.append(nl)
            else:
                break

        joined = ''.join(raw_buffer)
        m = CALLOUT_BLOCK_REGEX.match(joined)
        if not m:
            return None

        raw_type = m.group(1)
        fold_flag = m.group(2) or ''
        title = (m.group(3) or '').strip()
        canonical = CALLOUT_TYPE_MAP.get(raw_type.lower(), 'note')

        # Extract content lines, clean them properly
        raw_lines = joined.splitlines()[1:]  # Skip header line
        body_lines = []

        for raw in raw_lines:
            stripped = raw.lstrip()
            if stripped.startswith('>'):
                content = stripped[1:].lstrip()  # Remove '>' and leading space
                if content:
                    # Each line as a separate paragraph with final newline
                    body_lines.append(content + '\n\n')

        # Tokenize block
        parse_buffer = tokenize_block(
            body_lines, block_token._token_types, start_line=start_line_number + 1
        )

        return (parse_buffer, canonical, fold_flag, title)

    def __init__(self, read_result):
        parse_buffer, canonical_type, fold_flag, title = read_result
        super().__init__(parse_buffer)
        self.is_callout = True
        self.callout_type = canonical_type
        self.fold_flag = fold_flag
        self.callout_title = title or canonical_type.capitalize()


In [ ]:
#| export
class ObsidianAstRenderer(AstRenderer):
    def __init__(self,  **kwargs):
        super().__init__(Frontmatter, WikiLink, TagLink, AnyLink, ObsidianLink, ObsidianEmbed, ObsidianCallout, **kwargs)  # register custom token

    def render_obsidian_embed(self, token):
        return self.render_link(token)
    
    def render_obsidian_link(self, token):
        return self.render_link(token)

    def render_wiki_link(self, token):
        return self.render_link(token)

    def render_any_link(self, token):
        return self.render_link(token)

    def render_tag_link(self, token):
        return self.render_link(token)
    
    def render_obsidian_callout(self, token: ObsidianCallout):
        return self.render_quote(token)
    
    def render_frontmatter(self, token: Frontmatter) -> dict:
        # Just return dict so AST expansion shows structured metadata
        return token.children

In [ ]:
#| export

class ObsidianHTMLRenderer(HTMLRenderer):

    # VIDEO_EXTS = {".mp4", ".webm", ".ogg", ".mov", ".mkv"}
    # AUDIO_EXTS = {".mp3", ".wav", ".ogg", ".m4a", ".flac"}
    # IMAGE_EXTS = {".png", ".jpg", ".jpeg", ".gif", ".svg", ".webp"}
    # TEXTLIKE_EXTS = {".md", ".qmd", ".canvas", ".base"}

    def __init__(self, **kwargs):
        super().__init__(Frontmatter, WikiLink, TagLink, AnyLink, ObsidianLink, ObsidianEmbed, ObsidianCallout,  **kwargs)  # register custom tokens

    def render_obsidian_embed(self, token):
        target = token.target
        ext = token.ext
        fname = token.fname
        src = token.src
        alt = token.title or "" or (token.children[0].content if token.children else "")
        if ext in ExtensionTypes.IMAGE_EXTS: return self.render_image(token)
        if ext in ExtensionTypes.TEXTLIKE_EXTS: 
    
            # return f'<blockquote data-embed="{src}">{alt}</blockquote>'
            return (
            f'<blockquote class="note-embed-placeholder" '
            f'data-embed-url="{src}" '
            f'hx-get="{src}" '
            f'hx-trigger="revealed once" '
            f'hx-swap="outerHTML">'
            f'Loading {alt}…'
            f'</blockquote>'
        )
        if ext in ExtensionTypes.AUDIO_EXTS:
            mimetypes.guess_type
            # mime = f"audio/{ext[1:]}" if ext else "audio/mpeg"
            # mime = mimetypes.guess_type(fname, strict=False)[0]
            mime = guess_mime(fname)
            return f'<audio controls><source src="{src}" type="{mime}">Your browser does not support the audio tag.</audio>'
        if ext in ExtensionTypes.VIDEO_EXTS or "youtube.com" in src or "youtu.be" in src or "vimeo.com" in src:
            if "youtube.com/watch" in src or "youtu.be/" in src:
                parsed = urllib.parse.urlparse(src)
                qs = urllib.parse.parse_qs(parsed.query)
                vid = qs.get("v", [""])[0] if "v" in qs else parsed.path.split("/")[-1]
                return f'<iframe width="560" height="315" src="https://www.youtube.com/embed/{vid}" frameborder="0" allowfullscreen></iframe>'
            if "vimeo.com" in src:
                vid = src.rstrip("/").split("/")[-1]
                return f'<iframe src="https://player.vimeo.com/video/{vid}" width="640" height="360" frameborder="0" allowfullscreen></iframe>'
            # mime = f"video/{ext[1:]}" if ext else "video/mp4"
            # mime = mimetypes.guess_type(fname, strict=False)[0]
            mime = guess_mime(fname)
            return f'<video controls><source src="{src}" type="{mime}">Your browser does not support the video tag.</video>'
        return self.render_link(token)
    
    def render_obsidian_link(self, token):
        return self.render_link(token)
        
    def render_wiki_link(self, token):
        return self.render_link(token)

    def render_any_link(self, token):
        return self.render_link(token)
    
    def render_tag_link(self, token):
        """Render TagLink as a span with tag styling."""
        return self.render_link(token)

    def render_frontmatter(self, token: Frontmatter) -> str:
        if not token.children:
            return '<div class="frontmatter"></div>'

        rows = []
        for prop in token.children:  # each is a Properties token
            key_html = f"<th>{prop.key}</th>"
            value_html = ", ".join(self.render(child) for child in prop.children)
            rows.append(f"<tr>{key_html}<td>{value_html}</td></tr>")

        return (
            '<div class="frontmatter">\n'
            '<table>\n'
            + "\n".join(rows)
            + "\n</table>\n</div>"
        )

    def render_obsidian_callout(self, token: ObsidianCallout):
        if isinstance(token, ObsidianCallout) and getattr(token, 'is_callout', False):
            type_cls = token.callout_type or "note"  # fallback type
            collapsed_by_default = (token.fold_flag == "-")
            title = token.callout_title or type_cls.capitalize()

            # Render body HTML from children with line breaks
            body_html = ""
            # for child in token.children:
            #     print(child)
            #     body_html += f", {child}"
            body_html = "".join(self.render(child) for child in token.children)

            # Collapse container class - using 'plus' instead of type
            collapse_class = "collapse collapse-plus border"

            # Checkbox - no ID needed, just the class
            checked_attr = "" if collapsed_by_default else " checked"

            html = f"""
    <div class="{collapse_class}">
    <input type="checkbox" class="uk-input"{checked_attr}>
    <div class="collapse-title font-semibold">{title}</div>
    <blockquote class="collapse-content text-sm">
    {body_html}
    </blockquote>
    </div>
    """
            return html.strip()
        else:
            return super().render_quote(token)


    def render_block_code(self, token: block_token.BlockCode) -> str:
        # Handle ObservableJS blocks
        if token.language == "ojs":
            container_id = f"ojs-container-{uuid.uuid4().hex}"
            code = token.children[0].content if token.children else ""
            return f"""
<div id="{container_id}"></div>
<script type="ojs" data-container="{container_id}">
{code}
</script>
"""
        elif token.language == "base":
            return f"""<div></div>"""

        elif token.language == "plotlyjs":
            # Parse the JSON content inside the fence
            try:
                spec = json.loads(token.children[0].content)
            except json.JSONDecodeError:
                return f"<pre><code>{token.children[0].content}</code></pre>"

            div_id = f"plotly-{id(token)}"
            return f"""
<div id="{div_id}" style="width:100%;height:400px;"></div>
<script>
    Plotly.newPlot("{div_id}", {json.dumps(spec.get("data", []))}, {json.dumps(spec.get("layout", {}))});
</script>
"""

        elif token.language == "gradiolite":
            code = token.children[0].content if token.children else ""
            return f"""
<gradio-lite>
{code}
</gradio-lite>
"""

        elif token.language == 'apexchart':
            engine = ApexChartEngine(token)
            return engine.render_html()

        # You could add "base" or other languages here too.
        # Otherwise, fall back to the default implementation:
        return super().render_block_code(token)

In [ ]:
text = """
---
title: Product Mindset for RAG
tags:
collection:
  - "[[RAG]]"
---

This is fun. #impression
Test markdown [markdown](/markdown)
Check this "[[Note|My Alias]]".
2nd link ![Alias2](https://www.google.com/)
3rd link ![Alias3](Alias3)
4th link ![Alias4](Alias4.svg)

>[!Note]- Test
>This is a test
>my test

"""

In [ ]:
#| export

def get_links(root, link_types=(Link, AnyLink)):
    result = []
    if hasattr(root, 'children') and not isinstance(root, span_token.SpanToken) and root.children is not None:
        for c in root.children:
            if isinstance(c, link_types):
                result.append(c)
            elif hasattr(c, 'children'):
                result.extend(get_links(c, link_types=link_types))
    return result
    
class ObsidianPage(Page):

    # VIDEO_EXTS = [".mp4", ".webm", ".ogg", ".mov", ".mkv"]
    # AUDIO_EXTS = [".mp3", ".wav", ".ogg", ".m4a", ".flac"]
    # IMAGE_EXTS = [".png", ".jpg", ".jpeg", ".gif", ".svg", ".webp"]
    # TEXTLIKE_EXTS = [".md", ".qmd", ".canvas", ".base", '.ipynb']

    def __init__(self, text, fpath=None, created_time=None, modified_time=None, file_size=None):
        self._text = text
        # self._vault= vault if vault else pathlib.Path(".")
        self._fpath = pathlib.Path(fpath) if fpath else None

        self.created_time = created_time
        self.modified_time = modified_time
        self.file_size = file_size

    
    # @property
    # def file_extension(self):
    #     ext = self._fpath.suffix
    #     if ext in ExtensionTypes.all():
    #         return ext[1:]
    #     return None

    @classmethod
    def from_file_path(cls, file_path: Path):
        text: str = None
        file_path = pathlib.Path(file_path)
        if not file_path.is_file():
            # Handle cases where the path does not point to a regular file
            if not file_path.exists():
                text = f"[Error: File not found at path: {file_path}]"
            else:  # Path exists but is not a file (e.g., a directory, or a special file)
                text = f"[Error: Path exists but is not a regular file: {file_path}]"
            
  
        else:
            try:
                text = file_path.read_text(encoding="utf-8")
            except UnicodeDecodeError:
                text = f"[Error: Unable to decode file content at {file_path}. Please check encoding.]"
            except PermissionError:
                text = f"[Error: Permission denied to read file at {file_path}.]"
            except Exception as e:
                # Catch any other unexpected errors during file reading
                text = f"[Error: An unexpected error occurred while reading {file_path}. Reason: {e}]"

        s = file_path.stat(); s.st_size
        created_time = datetime.fromtimestamp(s.st_ctime_ns / 1e9, tz=timezone.utc)
        # mtime (last modification time in ns)
        modified_time = datetime.fromtimestamp(s.st_mtime_ns / 1e9, tz=timezone.utc)
        file_size = s.st_size
            
        return cls(text=text, 
                   fpath=file_path, 
                   created_time=created_time, 
                   modified_time=modified_time, 
                   file_size=file_size)


    # @property
    # def lockey(self):
    #     return str(self._fpath.relative_to(config.OBSIDIAN_VAULT)) if self._fpath else None

    @property
    def obsidian_url(self):
        return MountPaths.obsidian_url.with_qparams(vault=config.OBSIDIAN_VAULT.name, file=self.lockey)

    @property
    def doc(self):
        span_token.add_token(ObsidianEmbed)
        span_token.add_token(ObsidianLink)
        span_token.add_token(WikiLink)
        block_token.add_token(Frontmatter)
        block_token.add_token(ObsidianCallout)
        span_token.add_token(TagLink)
        return Document(self._text)

    @property
    def frontmatter(self)->Optional[Frontmatter]:
        for o in self.doc.children:
            if type(o).__name__ == 'Frontmatter': return o
        return None

    # @property
    # def parent(self):
    #     return self._fpath.parent.name
        
    def get_property(self, key='title'):
        if self.frontmatter:
            for p in self.frontmatter.children:
                if p.key == key: return p
        return None

    @property
    def file_tags(self):
        return get_links(self.frontmatter, link_types=(TagLink))

    @property
    def tags(self):
        return get_links(self.doc, link_types=(TagLink))

    # @property
    # def app_url(self):
    #     return MountPaths.open.to(file=self.lockey, title=self.title)

    @property
    def links(self):
        return get_links(self.doc)

    @property
    def title(self):
        p = self.get_property('title'); 
        if p:
            o = p.children[0]
            return o.content
        return self._fpath.stem if self._fpath else None

    @property
    def collection(self):
        collection = self.get_property('collection')
        return get_links(collection)
        # return [str(resolve_note_path(vault=config.OBSIDIAN_VAULT, 
        #            file=child.fname).relative_to(config.OBSIDIAN_VAULT))
        #            for child in collection.children
        #         ] if collection else None

    @property
    def html(self):
        with ObsidianHTMLRenderer() as renderer:
            # html = renderer.render(Document(lines))
            html = renderer.render(self.doc)
        return html

    @property
    def ast(self):
        with ObsidianAstRenderer() as renderer:
            # html = renderer.render(Document(lines))
            html = renderer.render(self.doc)
        return html

    # @property
    # def text(self):
    #     return self._text


    # @property
    # def checksum(self):
    #     return hashlib.md5(self._text.encode('utf-8')).hexdigest()

In [ ]:
# op = ObsidianPage(text)
# op.doc
# op.frontmatter
# op.title
# print(op.html)
# print(op.ast)
# fpath = "/Users/rahul1.saraf/rahuketu/programming/notesobs/journals/2025_09_15.md"
fpath = "/Users/rahul1.saraf/rahuketu/programming/notesobs/pages/@RahulSaraf.md"
fpath = "/Users/rahul1.saraf/rahuketu/programming/notesobs/Clippings/Fuck You, Show Me The Prompt. – 2.md"
fpath = "/Users/rahul1.saraf/rahuketu/programming/notesobs/pages/Product Mindset for RAG.md"
op = ObsidianPage.from_file_path(fpath)
# print(op.html)
op.title
op.file_extension
# l = op.get_property('collection').children[0]; l

# l.target
# l.alias
# l.title
# s = pathlib.Path(fpath).stat(); s.st_size

# s.st_ctime_ns
# s.st_mtime_ns
# ctime = datetime.fromtimestamp(s.st_ctime_ns / 1e9, tz=timezone.utc); ctime
# # mtime (last modification time in ns)

# mtime = datetime.fromtimestamp(s.st_mtime_ns / 1e9, tz=timezone.utc); mtime
# pathlib.Path(fpath).suffix

op.collection
o = op.doc.children[5]
# o.children[0].children[0]
op.lockey
op.obsidian_url

links = []

# def get_links(root):
#     result = []
#     if hasattr(root, 'children') and not isinstance(root, span_token.SpanToken) and root.children is not None:
#         for c in root.children:
#             if isinstance(c, (Link, AnyLink)):
#                 result.append(c)
#             elif hasattr(c, 'children'):
#                 result.extend(get_links(c))
#     return result
# for c in op.doc.children:

#     print(c)

# root = op.doc.children[0]; root
# # l = root.children[1].children[0]; print(l)
# # isinstance(l, Link) or isinstance(l, AnyLink)

# l2 = root.children[-1].children[0]
# isinstance(l2, Link)

# get_links(op.doc)
# op.frontmatter
# op.links
# op.tags
op.file_tags
op.links
# op.file_tags
# op.collection
# op.frontmatter
# op.file_tags
# s = get_links(op.doc); s

# for link in s: print(link.target)
op.app_url
op.parent

'pages'

In [ ]:
MountPaths.tags

<MountPaths.tags: '/tags'>

In [ ]:
# o = get_obsidianmd_ast(text); # o.children[0].children[2].children
# fm = o.children[1]
# fm.children
# cout = o.children[-1]
# c = cout.children[0];
# cout.callout_title, cout.callout_type,cout.fold_flag, c.children,  c.children[0].content


In [ ]:
# from mistletoe

In [ ]:
# with ObsidianHTMLRenderer() as renderer:
#     html = renderer.render(Document(lines))
    # html = renderer.render(Document(doc_real.read_text()))

In [ ]:
#| notest
# text = 'Check this "[[Note|My Alias]]".'
# html = markdown(text, renderer=ObsidianHTMLRenderer)
# print(html)

In [ ]:
#| notest
text = """
This is fun
Check this [[Note|My Alias]].
"""
o = ObsidianPage(text)
o.ast; o.ast
o.doc.children

[<mistletoe.block_token.Paragraph with 5 children line_number=2>]

In [ ]:
#| export

def print_ast(token, indent=0):
    pad = "  " * indent
    data = {k: v for k, v in getattr(token, "__dict__", {}).items() if not k.startswith("_")}
    print(f"{pad}{token.__class__.__name__}: {data}")
    
    # Safely iterate over children if present and iterable
    children = getattr(token, "children", None)
    if children:
        for child in children:
            print_ast(child, indent + 1)

In [ ]:
text = """
---
# title: Product Mindset for RAG
tags:
source: https://www.youtube.com/
collection:
  - "[[RAG]]"
---
This is fun #some
Check this [[Note|My Alias]].
My link [Alias](note)
2nd link ![Alias2](https://www.google.com/)
3rd link ![Alias3](Alias3)

> This is a test
> my test
"""
# doc = get_obsidianmd_ast(text)
# print_ast(doc)
# o = doc.children[0]

# get_frontmatter(doc)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()